In [ ]:
from competition_utils import *
from models.base_config import BaseConfig

# 제출 함수를 리스트로 정의 (submission_N_학번)
# 깃허브에는 최대 3개 업로드 데모시 하이퍼파라미터 튜닝 후 1개만 제출
submission_functions = ['submission_YOLOv8n', 'submission_YOLOv9t', 'submission_YOLOv10n', 'submission_YOLO11n', 'submission_YOLO12n',
                        'submission_YOLOXNano', 'submission_FLDetn', 'submission_HyperYOLOt', 'submission_LightweightOB', 'submission_YOGAn'] 
for submission_function in submission_functions:
    exec(f"from submissions.{submission_function} import {submission_function}")

# 분석 방향에 따라 iteration 수 수정
# 평가시에는 데모와 다른 스플릿 시드를 만들어 [1, 10]로 진행 예정
iterations = [1, 10] 
# 'Crown Detection'는 예제 데이터셋, 데모 및 평가시에는 'CV_Competition'
Dataset_Name = 'COCO_airplane' 

# 결과를 모아 하나의 csv 파일에 저장
results_df = pd.DataFrame(columns=[
    'Experiment Time', 'Iteration', 'Submission Function', 
    'IoU', 'Dice', 'Precision', 'Recall', 'Output Json Path',
])
csv_filename = os.path.join(BaseConfig.get_output_dir(), f"Evaluation_Results_{BaseConfig.get_experiment_time()}.csv")
json_foldername = "json"
vis_foldername = "vis"

os.makedirs(os.path.join(BaseConfig.get_output_dir(), json_foldername), exist_ok=True)
os.makedirs(os.path.join(BaseConfig.get_output_dir(), vis_foldername), exist_ok=True)

# 서로 다른 iteration, submission function으로 실험 진행
for iteration in range(iterations[0], iterations[1]+1):
    
    yaml_path = f'Datasets/{Dataset_Name}/data_iter_{iteration:02d}.yaml'
    
    for submission_function in submission_functions:
        
        ex_time = datetime.now().strftime('%y%m%d_%H%M%S')
        output_json_path = os.path.join(BaseConfig.get_output_dir(), json_foldername, f"{ex_time}_{submission_function}_Iter_{iteration}_detection_results.json")
        
        globals()[submission_function](yaml_path, output_json_path)
        
        labels_dir = f"Datasets/{Dataset_Name}/labels"  
        vis_output_dir = os.path.join(BaseConfig.get_output_dir(), vis_foldername, f"{ex_time}_visualization_results")
        image_level_result_path =os.path.join(BaseConfig.get_output_dir(), json_foldername, f"{ex_time}_{submission_function}_Iter_{iteration}_image_level_results.json")
        stats = eval_and_vis(yaml_path, output_json_path, labels_dir, image_level_result_path, vis_output_dir, vis=False) # 분석 방향에 따라 vis=True 설정
        new_row = {
            'Experiment Time': ex_time,
            'Iteration': iteration,
            'Submission Function': submission_function,
            'IoU': stats['IoU']['avg'],
            'Dice': stats['Dice']['avg'],
            'Precision': stats['Precision']['avg'],
            'Recall': stats['Recall']['avg'],
            'Output Json Path': output_json_path,
        }
        results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)
        results_df.to_csv(csv_filename, index=False)

train_data_iter_01: 100%|██████████| 670/670 [00:01<00:00, 616.09it/s]


✓  train_data_iter_01.json 저장


val_data_iter_01: 100%|██████████| 223/223 [00:00<00:00, 629.92it/s]


✓  val_data_iter_01.json 저장


test_data_iter_01: 100%|██████████| 224/224 [00:00<00:00, 636.04it/s]


...

2025-04-22 06:01:22 | INFO     | yolox.evaluators.coco_evaluator:259 - Evaluate in main process...
2025-04-22 06:01:22 | INFO     | yolox.evaluators.coco_evaluator:292 - Loading and preparing results...
2025-04-22 06:01:22 | INFO     | yolox.evaluators.coco_evaluator:292 - DONE (t=0.00s)
2025-04-22 06:01:22 | INFO     | pycocotools.coco:366 - creating index...
2025-04-22 06:01:22 | INFO     | pycocotools.coco:366 - index created!
2025-04-22 06:01:22 | INFO     | yolox.layers.jit_ops:111 - Load fast_cocoeval op in 0.002s.
2025-04-22 06:01:22 | INFO     | models.YOLOXNano.pkgs.yolox.core.trainer:381 - 
Average forward time: 0.92 ms, Average NMS time: 0.49 ms, Average inference time: 1.42 ms
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.062
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.207
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.026
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxD

100%|##########| 14/14 [00:00<00:00, 18.24it/s]
Using /home/gamejoongsa/.cache/torch_extensions/py310_cu128 as PyTorch extensions root...
No modifications detected for re-loaded extension module fast_cocoeval, skipping build step...
Loading extension module fast_cocoeval...
Running per image evaluation...
Evaluate annotation type *bbox*
COCOeval_opt.evaluate() finished in 0.01 seconds.
Accumulating evaluation results...
COCOeval_opt.accumulate() finished in 0.00 seconds.


2025-04-22 06:01:22 | INFO     | models.YOLOXNano.pkgs.yolox.core.trainer:200 - Training of experiment is done and the best AP is 8.72


YOLOX inference: 100%|##########| 224/224 [00:04<00:00, 47.15it/s]
✓ YOLOX 결과 저장 → output/250422_055904/json/250422_055904_submission_YOLOXNano_Iter_1_detection_results.json
결과가 output/250422_055904/json/250422_055904_submission_YOLOXNano_Iter_1_detection_results.json에 저장되었습니다.


/tmp/ipykernel_52454/3296525840.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)


In [ ]:
# 분석 방향에 따라 전체 결과를 모아 평균-표준편차-통계테스트 결과 저장 
keep_columns = ['Iteration','Submission Function']
keep_measures = ['IoU','Dice','Precision','Recall']  

reduction = 'Iteration'
row = 'Measure Type'
column = 'Submission Function'
reference_column = 'submission_YOLOv8n'

custom_fmt_template = '{mean_fmt} ± {std_fmt} {significance}'
significance_levels = [0.1, 0.05, 0.01]
decimal_places = 3
transpose = True
make_tables(csv_filename, keep_columns, keep_measures, reduction, row, column,  
            reference_column, custom_fmt_template, significance_levels, decimal_places, transpose)